In [130]:
from z3 import *
from astroid import *

In [131]:
# Dictionary to simulate parsed type annotations
types = {
    'x': 'int',
    'y': 'int',
    'z': 'int',
}

In [132]:
# Strings to simulate simple preconditions
pre = [
    'x ** 3 + y ** 3 == z ** 3',
#     'x > 0 and y > 0 and z > 0', as expected, this extra condition breaks Z3 so this is somehing we need to look out for!
    '(x >= 0 and y >= 0) and z >= 0',
]

In [133]:
# Here is what we want
x = Int('x')
y = Int('y')
z = Int('z')
solve(x ** 3 + y ** 3 == z ** 3, x >= 0, y >= 0, z >= 0)

[x = 0, z = 0, y = 0]


In [134]:
a, b = map(parse, pre)
a, b = a.body[0], b.body[0] # It is valid to assume preconditions are one line
a, b

(<Expr l.1 at 0x7fa8c0131df0>, <Expr l.1 at 0x7fa8c0131790>)

In [135]:
# "An Expr is any expression that does not have its value used or stored" from astroid docs, so a and b must be of type Expr
a, b = a.value, b.value
a, b

(<Compare l.1 at 0x7fa8c0131c70>, <BoolOp l.1 at 0x7fa8c01315e0>)

In [136]:
# It is important to determine what types <Expr>.value can take on
# For now, we will only consider Compare and BoolOp

In [137]:
a.left, a.ops

(<BinOp l.1 at 0x7fa8c0131e20>, [('==', <BinOp l.1 at 0x7fa8c0131190>)])

In [138]:
def apply_name(name, typ):
    """Set up the appropriate variable representation in Z3 based onn name an type (typ)."""
    # TODO: determine full list of supported types
    if typ == 'int':
        return Int(name)

def parse_compare(node, types):
    """Currently only supports comparisons with builtin arithmetic and boolean operations. 
    DOES NOT support builtin math functions or anything of the sort (yet)."""
    left, ops = node.left, node.ops
    if isinstance(left, BinOp):
        left = parse_bin_op(left, types)
    elif isinstance(left, Const):
        left = left.value
    elif isinstance(left, Name):
        left = apply_name(left.name, types[left.name])
    else:
        # Throw some error
        pass
    for item in ops:
        op, right = item
        if isinstance(right, BinOp):
            right = parse_bin_op(right, types)
        elif isinstance(right, Const):
            right = right.value
        elif isinstance(right, Name):
            right = apply_name(right.name, types[right.name])
        else:
            # Throw some error
            pass
        left = apply_bin_op(left, op, right)
    return left
            
def parse_bin_op(node, types):
    """Recurse on node.left, node.op, node.right."""
    # TODO: determine full list of what node.left or node.right can be
    left, op, right = node.left, node.op, node.right
    if isinstance(left, BinOp):
        left = parse_bin_op(left, types)
    elif isinstance(left, Const):
        left = left.value
    elif isinstance(left, Name):
        left = apply_name(left.name, types[left.name])
    else:
        # Throw some error
        pass
    
    if isinstance(right, BinOp):
        right = parse_bin_op(right, types)
    elif isinstance(right, Const):
        right = right.value
    elif isinstance(right, Name):
        right = apply_name(right.name, types[right.name])
    else:
        # Throw some error
        pass
    
    return apply_bin_op(left, op, right)
        
def apply_bin_op(left, op, right):
    """Given left, right, op, apply the binary operation."""
    # Todo: find out which binary operations are supported
    if op == '+':
        return left + right
    elif op == '**':
        return left ** right
    elif op == '==':
        return left == right
    elif op == '<=':
        return left <= right
    elif op == '>=':
        return left >= right

In [139]:
parse_bin_op(a.left, types)               

x**3 + y**3

In [140]:
solve(parse_compare(a, types))

[x = 0, z = 0, y = 0]
